In [ ]:
#import required libraries
from azure.ml import MLClient
from azure.ml.entities import CommandJob, Code, PipelineJob

In [ ]:
#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(subscription_id, resource_group, workspace)

In [ ]:
# Define the command job that will get run inside the pipeline
cmd1 = "echo "'"hello"'" && echo "'"world"'" > ${{outputs.world_output}}/world.txt"

job1_outputs = {"world_output":None}

job1 = CommandJob(
    command = cmd1,
    environment = 'AzureML-Minimal:18',
    compute = 'cpu-cluster',
    outputs=job1_outputs)

In [ ]:
cmd2 = "cat ${{inputs.world_input}}/world.txt"

job2_inputs = {'world_input': '${{jobs.job1.outputs.world_output}}'}

job2 = CommandJob(
    command=cmd2,
    environment = 'AzureML-Minimal:18',
    compute='cpu-cluster',
    inputs=job2_inputs)


In [ ]:
pipeline_job = PipelineJob(
    description='test',
    jobs={'job1':job1, 'job2': job2}
)

In [ ]:
#submit the pipeline job
returned_job = ml_client.jobs.create_or_update(pipeline_job)
#get a URL for the status of the job
returned_job.services["Studio"].endpoint